1. Реализовать простейший динамический массив, поддерживающий добавление элемента в конец массива при помощи метода append(e). Также должны поддерживаться следующие операции:
извлечение элемента по индексу;
установка нового значения по индексу;
получение текущей длины массива;
удаление элемента из любого места массива;
поддержка корректного вывода массива при помощи функции print.

In [4]:
class DynamicArray:
    def __init__(self):
        self.size = 0
        self.capacity = 1
        self.array = [None] * self.capacity

    def __getitem__(self, index):
        if index < 0 or index >= self.size:
            raise IndexError("Index out of range")
        return self.array[index]

    def __setitem__(self, index, value):
        if index < 0 or index >= self.size:
            raise IndexError("Index out of range")
        self.array[index] = value

    def append(self, value):
        if self.size == self.capacity:
            self._resize(2 * self.capacity)
        self.array[self.size] = value
        self.size += 1

    def delete(self, index):
        if index < 0 or index >= self.size:
            raise IndexError("Index out of range")
        for i in range(index, self.size - 1):
            self.array[i] = self.array[i + 1]
        self.size -= 1

    def _resize(self, new_capacity):
        new_array = [None] * new_capacity
        for i in range(self.size):
            new_array[i] = self.array[i]
        self.array = new_array
        self.capacity = new_capacity

    def length(self):
        return self.size

    def __str__(self):
        return "[" + ", ".join(str(self.array[i]) for i in range(self.size)) + "]"


# Пример использования:
arr = DynamicArray()
arr.append(1)
arr.append(2)
arr.append(3)
arr.append(4)

print(arr)            # Output: [1, 2, 3, 4]

arr.delete(2)
print(arr)            # Output: [1, 2, 4]


[1, 2, 3, 4]
[1, 2, 4]


2. Реализовать генератор, который возвращает значение поочередно извлекаемое из конца двух очередей (в качестве очереди используется deque из collections). Если очередь из которой извлекается элемент пуста - генератор заканчивает работу.

In [5]:
from collections import deque

def alternate_queue_extract(queue1, queue2):
    while queue1 or queue2:
        if queue2:
            yield queue2.pop()
        if queue1:
            yield queue1.pop()

# Пример использования:
q1 = deque([1, 2, 3])
q2 = deque([4, 5, 6, 7])

generator = alternate_queue_extract(q1, q2)
for value in generator:
    print(value)

7
3
6
2
5
1
4


3a) Класс Pump - в методе action() извлекает очередное значение из генератора и помещает значение в очередь (очередь передается в конструктор).

In [6]:
from collections import deque

In [14]:
class Pump:
    def __init__(self, queue, generator):
        self.queue = queue
        self.generator = generator

    def action(self):
        try:
            value = next(self.generator)  # Извлечение следующего значения из генератора
            self.queue.append(value)  # Помещение значения в очередь
        except StopIteration:
            pass

3b) Класс MultiAction - при вызове метода action() n раз вызвает метод action() класса, переданного в конструкторе. Число n также определяется в конструкторе.

In [15]:
class MultiAction:
    def __init__(self, cls, n):
        self.cls = cls
        self.n = n

    def action(self):
        for _ in range(self.n):
            self.cls.action()

3c) Класс MultiPump - в методе action() извлекает очередное значение из генератора и помещает значение в одну из очередей (очереди передается в конструкторе); очереди , в которые помещаются очередные значения, меняются по порядку.

In [16]:
class MultiPump:
    def __init__(self, queues, generator):
        self.queues = queues
        self.generator = generator
        self.current_queue_index = 0

    def action(self):
        try:
            value = next(self.generator)  # Извлечение следующего значения из генератора
            self.queues[self.current_queue_index].append(value)  # Помещение значения в текущую очередь
            self.current_queue_index = (self.current_queue_index + 1) % len(self.queues)  # Переход к следующей очереди
        except StopIteration:
            pass

In [28]:
gen = (i for i in range(1, 10))
queue1 = deque()
queue2 = deque()
pump1 = Pump(queue1, gen)
pump2 = Pump(queue2, gen)

multi_pump = MultiPump([queue1, queue2], gen)
multi_pump.action()
multi_pump.action()

multi_action = MultiAction(pump1, 3)
multi_action.action()

print(queue1)
print(queue2)

deque([1, 3, 4, 5])
deque([2])


4. При помощи GenFromQ, Pump реализовать систему опработки сообщений. Сообщения создаются генератором сообщений возвращающим случайным образорм одно из сообщений. Сообщения из трех генераторов закачиваются в три очереди при помощи классов Pump, далее при помощи GenFromQ и Pump объединяются в одну очередь и выводятся на экран (можно реализовать при помощи класса с action и вызываемого при помощи MultiPump).

In [32]:
import random
from collections import deque

# Генераторы сообщений
def message_generator1():
    messages = ['Hello!', 'Hey!', 'Salute!']
    while True:
        yield random.choice(messages)

def message_generator2():
    messages = ['Good morning!', 'Good afternoon!', 'Good evening!']
    while True:
        yield random.choice(messages)

def message_generator3():
    messages = ['Maybe we should take a walk?', 'How are you?', 'Keep smiling']
    while True:
        yield random.choice(messages)

# Очереди для хранения сообщений
queue1 = deque()
queue2 = deque()
queue3 = deque()
combined_queue = deque()

# Класс для загрузки сообщений в очередь
class Pump:
    def __init__(self, queue, generator):
        self.queue = queue
        self.generator = generator

    def action(self):
        try:
            value = next(self.generator)
            self.queue.append(value)
        except StopIteration:
            pass

# Класс для объединения сообщений из разных очередей
class GenFromQ:
    def __init__(self, queues, output_queue):
        self.queues = queues
        self.output_queue = output_queue

    def action(self):
        for q in self.queues:
            if q:
                message = q.popleft()
                self.output_queue.append(message)

# Загрузка сообщений в очереди
pump1 = Pump(queue1, message_generator1())
pump2 = Pump(queue2, message_generator2())
pump3 = Pump(queue3, message_generator3())

pump1.action()
pump2.action()
pump3.action()

# Объединение сообщений из разных очередей
gen_from_q = GenFromQ([queue1, queue2, queue3], combined_queue)
gen_from_q.action()

# Вывод сообщений на экран
while combined_queue:
    print(combined_queue.popleft())

Hey!
Good morning!
How are you?
